In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt



In [25]:
%%bash

filelist=("sam_vit_b_01ec64.pth" "sample_data")

pat=$(printf "^%s$" "${filelist[@]}")
pat=${pat:1}


ls | grep -Ev "$pat" | xargs rm -rf

In [26]:
!ls -R

.:


In [27]:
%%bash

# Variables
REPO_URL="https://github.com/LIMAMMohamedlimam/sammed-lite.git"
CLONE_DIR="temp_repo"
TARGET_DIR="./"
git clone "$REPO_URL" "$CLONE_DIR"

# Create target directory if it doesn't exist
mkdir -p "$TARGET_DIR"

# Copy all contents (including hidden files)
cp -r "$CLONE_DIR"/. "$TARGET_DIR"/

# Delete cloned repo directory
rm -rf "$CLONE_DIR"

echo "Done: copied repo content into $TARGET_DIR"

Done: copied repo content into ./


Cloning into 'temp_repo'...


In [28]:
!ls -R

.:
lite-sammed2d.py  SAMMed2D-lite.ipynb  segment_anything

./segment_anything:
automatic_mask_generator.py  __init__.py  predictor.py
build_sam.py		     modeling	  utils

./segment_anything/modeling:
common.py	  __init__.py	   prompt_encoder.py  transformer.py
image_encoder.py  mask_decoder.py  sam.py

./segment_anything/utils:
amg.py	__init__.py  onnx.py  transforms.py


In [29]:
from segment_anything.modeling import (
    Sam,
    ImageEncoderViT,
    MaskDecoder,
    PromptEncoder,
    TwoWayTransformer,
)

In [30]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## Adapter layer implementation


### Custom Transformer Block  (Adapter_layer injected)

In [ ]:

custom_image_encoder = ImageEncoderViT(
    transformer_block=adapter
)

TypeError: ImageEncoderViT.__init__() got an unexpected keyword argument 'transformer_block'

## Medical Image Dataset

In [32]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth


--2025-11-22 14:08:17--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.142.74, 108.157.142.16, 108.157.142.40, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.142.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  37.1MB/s    in 5.9s    

2025-11-22 14:08:23 (60.6 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]



In [34]:
sam_checkpoint = "sam_vit_b_01ec64.pth"

In [35]:
# test
from segment_anything.build_sam import sam_model_registry
sam = sam_model_registry["vit_b"](checkpoint=sam_checkpoint)



RuntimeError: Error(s) in loading state_dict for Sam:
	Missing key(s) in state_dict: "image_encoder.blocks.0.adapter_layer.norm.weight", "image_encoder.blocks.0.adapter_layer.norm.bias", "image_encoder.blocks.0.adapter_layer.channel.0.weight", "image_encoder.blocks.0.adapter_layer.channel.2.weight", "image_encoder.blocks.0.adapter_layer.spatial.0.weight", "image_encoder.blocks.0.adapter_layer.spatial.2.weight", "image_encoder.blocks.1.adapter_layer.norm.weight", "image_encoder.blocks.1.adapter_layer.norm.bias", "image_encoder.blocks.1.adapter_layer.channel.0.weight", "image_encoder.blocks.1.adapter_layer.channel.2.weight", "image_encoder.blocks.1.adapter_layer.spatial.0.weight", "image_encoder.blocks.1.adapter_layer.spatial.2.weight", "image_encoder.blocks.2.adapter_layer.norm.weight", "image_encoder.blocks.2.adapter_layer.norm.bias", "image_encoder.blocks.2.adapter_layer.channel.0.weight", "image_encoder.blocks.2.adapter_layer.channel.2.weight", "image_encoder.blocks.2.adapter_layer.s

In [38]:
for name,parm in sam.image_encoder.named_parameters() : 
    # if "Adapter" in name : 
        print(f"name : {name}")


name : pos_embed
name : patch_embed.proj.weight
name : patch_embed.proj.bias
name : blocks.0.norm1.weight
name : blocks.0.norm1.bias
name : blocks.0.attn.rel_pos_h
name : blocks.0.attn.rel_pos_w
name : blocks.0.attn.qkv.weight
name : blocks.0.attn.qkv.bias
name : blocks.0.attn.proj.weight
name : blocks.0.attn.proj.bias
name : blocks.0.norm2.weight
name : blocks.0.norm2.bias
name : blocks.0.mlp.lin1.weight
name : blocks.0.mlp.lin1.bias
name : blocks.0.mlp.lin2.weight
name : blocks.0.mlp.lin2.bias
name : blocks.1.norm1.weight
name : blocks.1.norm1.bias
name : blocks.1.attn.rel_pos_h
name : blocks.1.attn.rel_pos_w
name : blocks.1.attn.qkv.weight
name : blocks.1.attn.qkv.bias
name : blocks.1.attn.proj.weight
name : blocks.1.attn.proj.bias
name : blocks.1.norm2.weight
name : blocks.1.norm2.bias
name : blocks.1.mlp.lin1.weight
name : blocks.1.mlp.lin1.bias
name : blocks.1.mlp.lin2.weight
name : blocks.1.mlp.lin2.bias
name : blocks.2.norm1.weight
name : blocks.2.norm1.bias
name : blocks.2.att

## Model implementation SAM-Med2D-Lite

In [ ]:
class SAMMed2DLite(nn.Module):
    def __init__(
        self,
        sam_model : Sam,
        embed_dim=768,
    ):
        
        """Lite version of SAM-Med2D with adapter layers for medical image segmentation."""
        super().__init__()
        self.sam = sam_model

        #Freeze image encoder params except the adapter layer
        for name , params in self.sam.image_encoder.parameters() :
            if adap
